# Capstone Project - Week5 Submission (Ver. Soo)
### Class: Applied Data Science Capstone by IBM/Coursera

**Title: Opening a New Shopping Mall in Brisbane, Australia**

Objects:
1. Build a dataframe of neighborhoods in Brisbane, Australia by web scraping the data from Wikipedia page  
2. Get the geographical coordinates of the neighborhoods  
3. Obtain the venue data for the neighborhoods from Foursquare API  
4. Explore and cluster the neighborhoods  
5. Select the best cluster to open a new shopping mall  


## 1. Import libraries

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!pip install geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

     |████████████████████████████████| 92kB 7.0MB/s eta 0:00:011
Libraries imported.


## 2. Scrap data from Wikipedia page into a DataFrame

In [4]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Brisbane").text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,"► Acacia Ridge, Queensland‎ (3 P)"
1,"► Albion, Queensland‎ (6 P)"
2,"► Alderley, Queensland‎ (4 P)"
3,"► Annerley, Queensland‎ (9 P)"
4,"► Archerfield, Queensland‎ (5 P)"


In [9]:
# print the number of rows of the dataframe
kl_df.shape

(104, 1)

## 3. Get the geographical coordinates

In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Brisbane, Australia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [12]:
coords

[[-27.590409999999963, 153.02848000000006],
 [-27.42926003000722, 153.0422799744648],
 [-27.424489999999935, 152.99877000000004],
 [-27.513909999999953, 153.03114000000005],
 [-27.567169999999976, 153.01512000000002],
 [-27.565002, 152.9525115],
 [-27.445649999999944, 152.99196000000006],
 [-27.473419999999976, 152.99614000000008],
 [-27.322099999999978, 153.00995000000012],
 [-27.456199999999967, 153.06722000000002],
 [-27.374729999999943, 153.07810000000006],
 [-27.458009999999945, 152.9860500000001],
 [-27.563019999999938, 152.8859000000001],
 [-27.48903999999993, 153.12734000000012],
 [-27.34771999999998, 153.07143000000008],
 [-27.447599999999966, 153.03674000000012],
 [-27.364619999999945, 152.9909600000001],
 [-27.292929999999956, 153.06216000000006],
 [-27.46843999999993, 153.02334000000008],
 [-27.46843999999993, 153.02334000000008],
 [-27.492999999999938, 152.9108900000001],
 [-27.565002, 152.9525115],
 [-27.457649999999944, 153.0882200000001],
 [-27.489739999999983, 153.0993

In [13]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [15]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(104, 3)


,Neighborhood,Latitude,Longitude
0,"► Acacia Ridge, Queensland‎ (3 P)",-27.590410,153.028480
1,"► Albion, Queensland‎ (6 P)",-27.429260,153.042280
2,"► Alderley, Queensland‎ (4 P)",-27.424490,152.998770
3,"► Annerley, Queensland‎ (9 P)",-27.513910,153.031140
4,"► Archerfield, Queensland‎ (5 P)",-27.567170,153.015120
5,"► Ascot, Queensland‎ (10 P)",-27.565002,152.952512
6,"► Ashgrove, Queensland‎ (8 P)",-27.445650,152.991960
7,"► Auchenflower, Queensland‎ (1 C, 8 P)",-27.473420,152.996140
8,"► Bald Hills, Queensland‎ (4 P)",-27.322100,153.009950
9,"► Balmoral, Queensland‎ (3 P)",-27.456200,153.067220


In [17]:
# save the DataFrame as CSV file
kl_df.to_csv("Brisbane Data.csv", index=False)

## 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [18]:
# get the coordinates of Kuala Lumpur
address = 'Brisbane, Australia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brisbane, Australia {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brisbane, Australia -27.4689682, 153.0234991.


In [19]:
# create map of Brisbane using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [20]:
# save the map as HTML file
map_kl.save('map_kl.html')

## 5. Use the Foursquare API to explore the neighborhoods

In [21]:
# define Foursquare Credentials and Version
CLIENT_ID = '5LQXAGZJP1ZCHPMSRX3MLD2TMVAAWODLC2EGCACPQRI3Q1NI' # your Foursquare ID
CLIENT_SECRET = 'XQXNSDV5EUTNY5RB3R5EUDNHXYR25JKX3VTTQOLAUJ4ITNBC' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5LQXAGZJP1ZCHPMSRX3MLD2TMVAAWODLC2EGCACPQRI3Q1NI
CLIENT_SECRET:XQXNSDV5EUTNY5RB3R5EUDNHXYR25JKX3VTTQOLAUJ4ITNBC


#### Get the top 50 venues that are within a radius of 1000 meters.

In [22]:
radius = 1000
LIMIT = 50

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2369, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"► Acacia Ridge, Queensland‎ (3 P)",-27.59041,153.02848,IGA Acacia Ridge,-27.589866,153.027448,Grocery Store
1,"► Acacia Ridge, Queensland‎ (3 P)",-27.59041,153.02848,Subway,-27.585320,153.026710,Sandwich Place
2,"► Acacia Ridge, Queensland‎ (3 P)",-27.59041,153.02848,KFC,-27.587570,153.027600,Fast Food Restaurant
3,"► Acacia Ridge, Queensland‎ (3 P)",-27.59041,153.02848,McDonald's,-27.589040,153.027000,Fast Food Restaurant
4,"► Acacia Ridge, Queensland‎ (3 P)",-27.59041,153.02848,Acacia Ridge Shopping Centre,-27.590135,153.027384,Food & Drink Shop


#### Number of  venues returned for each neighorhood

In [24]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"► Acacia Ridge, Queensland‎ (3 P)",7,7,7,7,7,7
"► Albion, Queensland‎ (6 P)",28,28,28,28,28,28
"► Alderley, Queensland‎ (4 P)",19,19,19,19,19,19
"► Annerley, Queensland‎ (9 P)",35,35,35,35,35,35
"► Archerfield, Queensland‎ (5 P)",5,5,5,5,5,5
"► Ascot, Queensland‎ (10 P)",13,13,13,13,13,13
"► Ashgrove, Queensland‎ (8 P)",24,24,24,24,24,24
"► Auchenflower, Queensland‎ (1 C, 8 P)",50,50,50,50,50,50
"► Bald Hills, Queensland‎ (4 P)",7,7,7,7,7,7


#### Number of unique categories

In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 228 uniques categories.


In [26]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Grocery Store', 'Sandwich Place', 'Fast Food Restaurant',
       'Food & Drink Shop', 'Mexican Restaurant',
       'Construction & Landscaping', 'Bakery', 'Indian Restaurant',
       'Seafood Restaurant', 'Café', 'Breakfast Spot',
       'Turkish Restaurant', 'Scenic Lookout', 'Cricket Ground',
       'Comedy Club', 'Train Station', 'Park', 'Gym', 'Recording Studio',
       'Sushi Restaurant', 'Bus Station', 'Convenience Store',
       'Burrito Place', 'Japanese Restaurant', 'Laundromat',
       'Supermarket', 'Athletics & Sports', 'Pool', 'Thai Restaurant',
       'Pizza Place', 'Gas Station', 'Volleyball Court', 'Liquor Store',
       'Asian Restaurant', 'Kebab Restaurant', 'Australian Restaurant',
       'Coffee Shop', 'Vietnamese Restaurant', 'Burger Joint',
       'Sporting Goods Shop', 'Gym / Fitness Center', 'Bus Stop',
       'Shopping Mall', 'Pharmacy', 'Airport', 'Thrift / Vintage Store',
       'Bar', 'Chinese Restaurant', 'Massage Studio', 'Pet Store'],
      dtype=

In [27]:
# check if the results contain "Bakery"
"Bakery" in venues_df['VenueCategory'].unique()

True

## 6. Analyze Each Neighborhood

In [28]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2369, 229)


,Neighborhoods,Afghan Restaurant,Airport,Airport Service,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,RV Park,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Sporting Goods Shop,Sports Club,Stables,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,"► Acacia Ridge, Queensland‎ (3 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"► Acacia Ridge, Queensland‎ (3 P)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(103, 229)


,Neighborhoods,Afghan Restaurant,Airport,Airport Service,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Store,Big Box Store,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hostel,Hot Dog Joint,Hotel,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Noodle House,Office,Organic Grocery,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,RV Park,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Sporting Goods Shop,Sports Club,Stables,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Water Park,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,"► Acacia Ridge, Queensland‎ (3 P)",0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.142857,0.000000,0.0,0.00000,0.000000,0.000000,0.00,0.000000,0.00000

In [30]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

34

#### DataFrame for Bakery data only

In [31]:
kl_mall = kl_grouped[["Neighborhoods","Bakery"]]
kl_mall.head()

,Neighborhoods,Bakery
0,"► Acacia Ridge, Queensland‎ (3 P)",0.000000
1,"► Albion, Queensland‎ (6 P)",0.035714
2,"► Alderley, Queensland‎ (4 P)",0.000000
3,"► Annerley, Queensland‎ (9 P)",0.028571
4,"► Archerfield, Queensland‎ (5 P)",0.000000


## 7. Cluster Neighborhoods

In [32]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 0], dtype=int32)

In [33]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [34]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Bakery,Cluster Labels
0,"► Acacia Ridge, Queensland‎ (3 P)",0.000000,0
1,"► Albion, Queensland‎ (6 P)",0.035714,1
2,"► Alderley, Queensland‎ (4 P)",0.000000,0
3,"► Annerley, Queensland‎ (9 P)",0.028571,0
4,"► Archerfield, Queensland‎ (5 P)",0.000000,0


In [35]:
# Add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(103, 5)


,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,"► Acacia Ridge, Queensland‎ (3 P)",0.000000,0,-27.59041,153.02848
1,"► Albion, Queensland‎ (6 P)",0.035714,1,-27.42926,153.04228
2,"► Alderley, Queensland‎ (4 P)",0.000000,0,-27.42449,152.99877
3,"► Annerley, Queensland‎ (9 P)",0.028571,0,-27.51391,153.03114
4,"► Archerfield, Queensland‎ (5 P)",0.000000,0,-27.56717,153.01512


In [36]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(103, 5)


,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,"► Acacia Ridge, Queensland‎ (3 P)",0.000000,0,-27.590410,153.028480
68,"► Nudgee Beach, Queensland‎ (2 P)",0.000000,0,-27.348830,153.105180
67,"► Norman Park, Queensland‎ (6 P)",0.000000,0,-27.480033,153.078572
65,"► New Farm, Queensland‎ (20 P)",0.020000,0,-27.465890,153.044510
64,"► Mount Ommaney, Queensland‎ (2 P)",0.000000,0,-27.544310,152.931650
63,"► Mount Gravatt East, Queensland‎ (3 P)",0.000000,0,-27.526740,153.089920
62,"► Mount Coot-tha, Queensland‎ (5 P)",0.000000,0,-27.478210,152.965920
59,"► Moggill, Queensland‎ (3 P)",0.000000,0,-27.568640,152.880670
57,"► Milton, Queensland‎ (1 C, 14 P)",0.000000,0,-27.468070,153.006790
56,"► Manly, Queensland‎ (6 P)",0.000000,0,-27.454210,153.186210


#### visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 8. Results

#### Cluster 0

In [39]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
0,"► Acacia Ridge, Queensland‎ (3 P)",0.000000,0,-27.590410,153.028480
68,"► Nudgee Beach, Queensland‎ (2 P)",0.000000,0,-27.348830,153.105180
67,"► Norman Park, Queensland‎ (6 P)",0.000000,0,-27.480033,153.078572
65,"► New Farm, Queensland‎ (20 P)",0.020000,0,-27.465890,153.044510
64,"► Mount Ommaney, Queensland‎ (2 P)",0.000000,0,-27.544310,152.931650
63,"► Mount Gravatt East, Queensland‎ (3 P)",0.000000,0,-27.526740,153.089920
62,"► Mount Coot-tha, Queensland‎ (5 P)",0.000000,0,-27.478210,152.965920
59,"► Moggill, Queensland‎ (3 P)",0.000000,0,-27.568640,152.880670
57,"► Milton, Queensland‎ (1 C, 14 P)",0.000000,0,-27.468070,153.006790
56,"► Manly, Queensland‎ (6 P)",0.000000,0,-27.454210,153.186210


#### Cluster 1

In [40]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
94,"► Wacol, Queensland‎ (10 P)",0.076923,1,-27.565002,152.952512
96,"► Wilston, Queensland‎ (3 P)",0.068966,1,-27.432200,153.019630
92,"► Toowong‎ (1 C, 25 P)",0.040000,1,-27.478510,152.985620
5,"► Ascot, Queensland‎ (10 P)",0.076923,1,-27.565002,152.952512
6,"► Ashgrove, Queensland‎ (8 P)",0.041667,1,-27.445650,152.991960
89,"► The Gap, Queensland‎ (5 P)",0.055556,1,-27.444900,152.952910
98,► Woolloongabba‎ (24 P),0.040000,1,-27.490950,153.035340
47,"► Ithaca, Queensland‎ (7 P)",0.076923,1,-27.565245,152.953137
87,"► Taringa, Queensland‎ (6 P)",0.038462,1,-27.491700,152.981500
86,"► Stafford, Queensland‎ (2 P)",0.066667,1,-27.409070,153.006780


#### Cluster 2

In [41]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Bakery,Cluster Labels,Latitude,Longitude
17,"► Brighton, Queensland‎ (2 P)",0.250000,2,-27.29293,153.06216
13,"► Belmont, Queensland‎ (2 P)",0.166667,2,-27.48904,153.12734


**Discussion:**

In terms of facility cencentration, most of the bakery are placed in the central area of Brisbane city, with the highest number in cluster 2. On the other hand, cluster 0 has very low number of Bakery in the neighborhoods. This represents a great opportunity and high potential areas to open new bakery as there is very little to no competition from existing bakeries. Meanwhile, Bakeries in cluster 2 are likely suffering from intense competition due to oversupply and high concentration. From another perspective, this also shows that the oversupply of Bakeries mostly happened in the Asian town. Therefore, this project recommends property developers to capitalize on these findings to open new bakeries in neighborhoods in cluster 0 with little to no competition. However, this is only an analysis based on the concentration of facilities, and additional variables such as population density or change of population density must be considered. Therefore, this report recommends cluster 0 in opening a new bakery but highlights the need for further research.